In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoCharacterMapper, ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:01, 205.20it/s]
251it [00:00, 319.93it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x1a2a760be0>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [18]:
vocab = Vocabulary()

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [19]:
from allennlp.data.iterators import BucketIterator

In [20]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [21]:
iterator.index_with(vocab)

### Read sample

In [22]:
batch = next(iter(iterator(train_ds)))

In [23]:
batch

{'tokens': {'tokens': tensor([[[259,  35, 260,  ..., 261, 261, 261],
           [259,  90, 102,  ..., 261, 261, 261],
           [259, 117, 105,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  74, 260,  ..., 261, 261, 261],
           [259,  71,  74,  ..., 261, 261, 261],
           [259,  74,  85,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  71, 115,  ..., 261, 261, 261],
           [259, 100, 109,  ..., 261, 261, 261],
           [259,  74, 260,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          ...,
  
          [[259,  66

In [24]:
batch["tokens"]["tokens"]

tensor([[[259,  35, 260,  ..., 261, 261, 261],
         [259,  90, 102,  ..., 261, 261, 261],
         [259, 117, 105,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  74, 260,  ..., 261, 261, 261],
         [259,  71,  74,  ..., 261, 261, 261],
         [259,  74,  85,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  71, 115,  ..., 261, 261, 261],
         [259, 100, 109,  ..., 261, 261, 261],
         [259,  74, 260,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[259,  66, 260,  ..., 261, 261, 261],
         [259, 115, 102,  ..., 261, 261, 261]

In [25]:
batch["tokens"]["tokens"].shape

torch.Size([64, 45, 50])

# Prepare Model

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

In [27]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [28]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

02/03/2019 17:30:50 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [29]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [30]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [31]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [32]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [33]:
tokens = batch["tokens"]
labels = batch

In [34]:
tokens

{'tokens': tensor([[[259,  35, 260,  ..., 261, 261, 261],
          [259,  90, 102,  ..., 261, 261, 261],
          [259, 117, 105,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  74, 260,  ..., 261, 261, 261],
          [259,  71,  74,  ..., 261, 261, 261],
          [259,  74,  85,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  71, 115,  ..., 261, 261, 261],
          [259, 100, 109,  ..., 261, 261, 261],
          [259,  74, 260,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         ...,
 
         [[259,  66, 260,  ..., 261, 261, 261],
        

In [35]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [36]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 6.0163e-02,  6.4747e-02,  2.6297e-01, -1.6403e-02,  1.9168e-02,
         -1.6628e-01],
        [ 9.5824e-02,  8.7803e-02,  2.4718e-02,  3.7297e-02,  6.7119e-02,
         -2.0273e-01],
        [-6.7574e-02,  1.5751e-01,  1.0635e-01, -4.4015e-03,  1.4845e-02,
         -3.1285e-02],
        [-1.0367e-03, -5.3197e-02, -1.2650e-02,  2.0672e-01,  9.8254e-02,
         -7.4023e-02],
        [-9.9698e-02,  5.6070e-02,  6.4787e-02, -1.1896e-01,  5.9189e-03,
         -1.0728e-01],
        [-1.6731e-01,  1.9377e-02,  1.6526e-01, -8.5814e-02,  1.1862e-01,
         -2.2048e-01],
        [ 6.0305e-02,  6.5591e-02, -4.8577e-02, -8.1078e-02,  1.0256e-01,
         -1.1554e-01],
        [ 2.7622e-02, -1.1867e-02,  9.7557e-02,  5.0214e-02,  2.7249e-02,
         -1.0539e-01],
        [ 6.2683e-02, -1.0014e-01,  2.0654e-01,  1.2998e-01,  9.3514e-02,
         -4.2657e-03],
        [-7.0279e-02, -6.9139e-02, -3.1017e-02, -2.8966e-02,  7.2114e-02,
          1.9947e-02],
        [-1.1726e-01,  1.3738e

In [37]:
model(**batch)

{'class_logits': tensor([[-3.4829e-02,  5.2497e-02,  2.2299e-01, -1.8890e-01, -2.0429e-03,
          -7.4547e-02],
         [ 1.3691e-01,  2.1239e-02,  2.3981e-02,  2.1327e-02,  2.0890e-02,
          -1.8907e-01],
         [-1.7149e-01,  8.6617e-03,  4.0481e-02, -7.7425e-02, -4.0966e-02,
          -1.1208e-01],
         [-3.6241e-02, -6.5557e-02, -7.7019e-02,  4.4819e-02,  3.3430e-02,
          -2.4298e-01],
         [-8.3382e-02,  7.1285e-02,  1.7028e-01, -6.6278e-02,  1.2807e-01,
          -1.9301e-01],
         [-1.2943e-01, -8.1146e-02,  7.5968e-02, -3.1295e-02, -7.5692e-02,
          -1.4631e-01],
         [ 1.2931e-02,  1.1064e-01,  3.3163e-02, -3.6950e-02, -2.4099e-02,
          -1.3883e-01],
         [ 5.2409e-02,  1.0081e-01,  2.1262e-02, -8.8295e-02, -2.6079e-02,
          -7.9742e-02],
         [ 2.2081e-02, -1.3706e-01,  1.7494e-01,  5.8705e-02,  3.2575e-02,
          -6.8659e-02],
         [ 7.9164e-02, -6.5224e-02,  7.3719e-02, -5.4167e-02,  1.5824e-01,
           8.0593e

In [38]:
loss = model(**batch)["loss"]

In [39]:
loss

tensor(0.6972, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [40]:
loss.backward()

In [41]:
[x.grad for x in list(model.encoder.parameters())]

[tensor([[ 1.3974e-05,  1.1389e-05,  4.9973e-05,  ..., -7.9098e-05,
           4.7838e-05, -1.9684e-05],
         [-1.2166e-04, -6.3747e-05, -1.2056e-04,  ..., -1.2964e-04,
           1.3664e-04, -6.8199e-05],
         [-1.3146e-04,  9.1971e-05,  3.6641e-05,  ...,  6.4251e-05,
           1.5939e-04,  8.8090e-05],
         ...,
         [ 2.5123e-04, -4.7318e-05,  5.9469e-05,  ..., -1.0526e-04,
          -2.5753e-04,  6.4145e-05],
         [-5.1145e-05, -1.0931e-05,  6.5940e-05,  ..., -2.3286e-05,
           9.6785e-05,  5.9618e-06],
         [ 1.0809e-04,  1.0349e-05,  4.6627e-05,  ..., -3.2571e-05,
          -1.3261e-04,  1.2602e-04]]),
 tensor([[ 1.0903e-05,  1.2475e-05, -2.9365e-06,  ..., -1.1445e-05,
           1.9748e-05, -8.9563e-06],
         [-1.5088e-05,  2.9193e-05, -5.3877e-05,  ..., -1.3567e-05,
          -4.9185e-06,  4.6052e-06],
         [ 7.9974e-06,  3.5755e-07, -6.2460e-05,  ..., -2.5084e-05,
           5.9656e-06,  3.7861e-05],
         ...,
         [-6.7549e-06,  1

# Train

In [42]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [43]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [44]:
metrics = trainer.train()

02/03/2019 17:31:08 - INFO - allennlp.training.trainer -   Beginning training.
02/03/2019 17:31:08 - INFO - allennlp.training.trainer -   Epoch 0/1
02/03/2019 17:31:08 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1845.4528
02/03/2019 17:31:08 - INFO - allennlp.training.trainer -   Training
loss: 0.6843 ||: 100%|██████████| 5/5 [00:27<00:00,  5.72s/it]
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -                     Training |  Validation
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1845.453  |       N/A
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   loss          |     0.684  |       N/A
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:27
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:27
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   Epoch 1/1
02/03/2019 17:31:36 - INFO - allennlp.training.trainer -   Peak CPU memory 